# Environment


In [1]:
# import libraries
import os
import pandas as pd  # for storing text and embeddings data
from openai import OpenAI  # for calling the OpenAI API
from openai.types.chat import ChatCompletionMessageParam
import tiktoken  # for counting tokens
from utils.embeddings_utils import cosine_similarity

# models
EMBEDDING_MODEL = "text-embedding-3-large"
GPT_MODEL = "ft:gpt-3.5-turbo-0125:personal:transfer-0613:9ZWWwDJS"

In [2]:
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# Intent Parsing


In [3]:
system_prompt = """
    You will be presented with a user intent, and your task is to extract the useful information and output it in JSON format as shown below:

    ```json
    {
    "action": "transfer",
    "chain": "Ethereum",
    "amount": "12",
    "token": "ETH",
    "receiver": "trump.eth"
    }
    ```

    If the intent is unknown, just say “I don't know”.
"""


def parse(intent: str):
    """
    Parse the user intent and return the extracted information in JSON format.
    """
    messages: list[ChatCompletionMessageParam] = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": intent},
    ]
    response = client.chat.completions.create(
        model=GPT_MODEL, messages=messages, temperature=0.7, max_tokens=500
    )
    return response.choices[0].message.content

In [4]:
parse("I want to transfer 12 ETH to trump.eth on Blast.")

'[{"action": "transfer", "chain": "Blast", "amount": "12", "token": "ETH", "receiver": "trump.eth"}]'

In [5]:
parse("Send 12 DOGE to 0x436f795B64E23E6cE7792af4923A68AFD3967952 on Ethereum.")

'[{"action": "transfer", "chain": "Ethereum", "amount": "12", "token": "DOGE", "receiver": "0x436f795B64E23E6cE7792af4923A68AFD3967952"}]'

In [6]:
parse("Send 12 PEPE and 10 USDC to apple.eth on Optimism.")

'[{"action": "transfer", "chain": "Optimism", "amount": "12", "token": "PEPE", "receiver": "apple.eth"}, {"action": "transfer", "chain": "Optimism", "amount": "10", "token": "USDC", "receiver": "apple.eth"}]'

In [7]:
parse("Send 12 PEPE to apple.eth and 10 USDT to banana.eth on Gnosis.")

'[{"action": "transfer", "chain": "Gnosis", "amount": "12", "token": "PEPE", "receiver": "apple.eth"}, {"action": "transfer", "chain": "Gnosis", "amount": "10", "token": "USDT", "receiver": "banana.eth"}]'

In [8]:
parse("Split 10 USDC and send equally to biden.eth and trump.eth on Ethereum.")

'[{"action": "transfer", "chain": "Ethereum", "amount": "5", "token": "USDC", "receiver": "biden.eth"}, {"action": "transfer", "chain": "Ethereum", "amount": "5", "token": "USDC", "receiver": "trump.eth"}]'

In [9]:
parse(
    "Send 10 USDC to vitalik.eth on Ethereum, 10 USDC to armstrong.eth on Base and 4 USDC to cz-binance.eth on BNB Chain."
)

'[{"action": "transfer", "chain": "Ethereum", "amount": "10", "token": "USDC", "receiver": "vitalik.eth"}, {"action": "transfer", "chain": "Base", "amount": "10", "token": "USDC", "receiver": "armstrong.eth"}, {"action": "transfer", "chain": "BNB", "amount": "4", "token": "USDC", "receiver": "cz-binance.eth"}]'

In [10]:
parse("Send 10 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48 to vitalik.eth on Ethereum")

'[{"action": "transfer", "chain": "Ethereum", "amount": "10", "token": "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48", "receiver": "vitalik.eth"}]'

# Outputs Generation


In [11]:
from utils.action_utils import evaluate_response
import json


def print_pretty_result(result: str):
    """
    Print the parsed result and the actions in a pretty format for demo.
    """
    print(f"parsed: {result}\n====================")
    actions = evaluate_response(result)
    # Example of the parsed actions
    parsed_actions = [json.loads(action) for action in actions]
    print(f"actions: {json.dumps(parsed_actions, indent=4)}")

In [12]:
from utils.action_utils import evaluate_response
import json

parsed = parse(
    "I want to transfer 12 USDC to 0x9fD042a18E90Ce326073fA70F111DC9D798D9a52 on Ethereum."
)
print_pretty_result(parsed)

parsed: [{"action": "transfer", "chain": "Ethereum", "amount": "12", "token": "USDC", "receiver": "0x9fD042a18E90Ce326073fA70F111DC9D798D9a52"}]
actions: [
    {
        "description": "Transfer 12 USDC to 0x9fD042...8D9a52 on Ethereum",
        "to": "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48",
        "value": "0",
        "data": "0xa9059cbb0000000000000000000000009fd042a18e90ce326073fa70f111dc9d798d9a520000000000000000000000000000000000000000000000000000000000b71b00"
    }
]


In [13]:
parsed = parse("Send 12 DOGE to 0x123 on Ethereum.")
print_pretty_result(parsed)

parsed: [{"action": "transfer", "chain": "Ethereum", "amount": "12", "token": "DOGE", "receiver": "0x123"}]
actions: [
    {
        "type": "InvalidArgumentError",
        "error": "Invalid recipient address: 0x123",
        "code": 400
    }
]


In [14]:
parsed = parse("Send 0.5 USDC to vitalik.eth on Ethereum.")
print_pretty_result(parsed)

parsed: [{"action": "transfer", "chain": "Ethereum", "amount": "0.5", "token": "USDC", "receiver": "vitalik.eth"}]
actions: [
    {
        "description": "Transfer 0.5 USDC to 0xd8dA6B...A96045 on Ethereum",
        "to": "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48",
        "value": "0",
        "data": "0xa9059cbb000000000000000000000000d8da6bf26964af9d7eed9e03e53415d37aa96045000000000000000000000000000000000000000000000000000000000007a120"
    }
]


In [15]:
parsed = parse(
    "Send 0.5 USDC to vitalik.eth and send 100 USDT to doge.eth on Ethereum."
)
print_pretty_result(parsed)

parsed: [{"action": "transfer", "chain": "Ethereum", "amount": "0.5", "token": "USDC", "receiver": "vitalik.eth"}, {"action": "transfer", "chain": "Ethereum", "amount": "100", "token": "USDT", "receiver": "doge.eth"}]
actions: [
    {
        "description": "Transfer 0.5 USDC to 0xd8dA6B...A96045 on Ethereum",
        "to": "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48",
        "value": "0",
        "data": "0xa9059cbb000000000000000000000000d8da6bf26964af9d7eed9e03e53415d37aa96045000000000000000000000000000000000000000000000000000000000007a120"
    },
    {
        "description": "Transfer 100 USDT to 0xDDd396...076364 on Ethereum",
        "to": "0xdac17f958d2ee523a2206206994597c13d831ec7",
        "value": "0",
        "data": "0xa9059cbb000000000000000000000000ddd3964d75d59b6b6d5c31eb313bba5ebf0763640000000000000000000000000000000000000000000000000000000005f5e100"
    }
]


In [16]:
parsed = parse("Send 0.5 PEPE to doge.eth on Base.")
print_pretty_result(parsed)

parsed: [{"action": "transfer", "chain": "Base", "amount": "0.5", "token": "PEPE", "receiver": "doge.eth"}]
actions: [
    {
        "type": "TokenNotFoundError",
        "error": "PEPE is not found on Base.",
        "code": 401
    }
]
